In [ ]:
%%javascript
(function(){
  function clickConnect(){
    const host = document.querySelector('colab-connect-button');
    const btn = host && host.shadowRoot ? host.shadowRoot.querySelector('#connect') : null;
    (btn || document.querySelector('#connect'))?.click();
    console.log('Colab ping');
  }
  setInterval(clickConnect, 60000);
  console.log('✅ Anti-idle ativo');
})();

In [ ]:
!apt -y update -qq
!apt -y install -qq aria2 rustc
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.33.2
!pip install tokenizers==0.13.3 --no-build-isolation --prefer-binary
!pip install diffusers xformers accelerate safetensors

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui /content/stable-diffusion-webui

In [ ]:
!mkdir -p /content/stable-diffusion-webui/models/Lora
!ln -s /content/drive/MyDrive/SD_LoRAs /content/stable-diffusion-webui/models/Lora

In [ ]:
%cd /content/stable-diffusion-webui
!COMMANDLINE_ARGS="--share --disable-safe-unpickle" \
  REQS_FILE="requirements.txt" \
  python launch.py